1. 训练Baseline model
2. 对Baseline model进行prune, 得到Baseline_pruned model
3. 训练Baseline_pruned model
4. 对Baseline_pruned model进行post-training quantization, 得到Baseline_pruned_quantized model
5. 对比Baseline, Baseline_pruned, Baseline_pruned_quantized的test acc, model size, elapsed time

# Setup

In [1]:
import tempfile
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import zipfile
%load_ext tensorboard

# Train a model for MNIST without pruning

In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=4,
    validation_split=0.1
)

Epoch 1/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.3148 - accuracy: 0.9111 - val_loss: 0.1488 - val_accuracy: 0.9610
Epoch 2/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.1443 - accuracy: 0.9586 - val_loss: 0.1074 - val_accuracy: 0.9688
Epoch 3/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.1025 - accuracy: 0.9707 - val_loss: 0.0821 - val_accuracy: 0.9762
Epoch 4/4
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0817 - accuracy: 0.9766 - val_loss: 0.0729 - val_accuracy: 0.9800


## Evaluate baseline test accuracy and save the model for later usage

In [3]:
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Baseline test accuracy: ', baseline_model_accuracy)

keras_file = './baseline.h5'
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to: ', keras_file)

Baseline test accuracy:  0.9753999710083008
Saved baseline model to:  ./baseline.h5


# Fine-tune pre-trained model with pruning

## Define the model

You will apply pruning to the whole model and see this in the model summary

in this example, you start the model with 50% sparsity(50% zeros in weights) and end with 80% sparsity

in the comprehensive guide, you can see how to prune some layers for model accuracy improvements

In [4]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs
batch_size = 128
epochs = 2
validation_split = 0.1

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                            final_sparsity=0.80,
                                                            begin_step=0,
                                                            end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# prune_low_magnitude requires a recompile
model_for_pruning.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


## Train and evaluate the model against baseline

Fine tune with pruning for two epochs

tfmot.sparsity.keras.UpdatePruningStep is required during training

tfmot.sparsity.keras.PruningSummaries provides logs for tracking progress and debugging

In [5]:
logdir = './logs'

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)
]

model_for_pruning.fit(train_images, train_labels,
                     batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                     callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 3s 7ms/step - loss: 0.0907 - accuracy: 0.9747 - val_loss: 0.0983 - val_accuracy: 0.9767
Epoch 2/2
422/422 [==============================] - 3s 6ms/step - loss: 0.1052 - accuracy: 0.9705 - val_loss: 0.0910 - val_accuracy: 0.9765


## For this example, there is minimal loss in test accuracy after pruning, compared to the baseline

In [6]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(test_images, test_labels, verbose=0)
print('Baseline test accuracy: ', baseline_model_accuracy)
print('Pruned test accuracy: ', model_for_pruning_accuracy)

Baseline test accuracy:  0.9753999710083008
Pruned test accuracy:  0.9713000059127808


## The logs show the progression of sparsity on a per-layer basis

In [7]:
%tensorboard --logdir={logdir}

Reusing TensorBoard on port 6006 (pid 28315), started 1:01:50 ago. (Use '!kill 28315' to kill it.)

# Create 3x smaller models from pruning

both tfmot.sparsity.keras.strip_pruning and applying a standard compression algorithm(via gzip) are necessary to see the compression benefits of pruning

## First, create a compressible model for tensorflow

In [8]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

pruned_keras_file = './baseline_pruned.h5'
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to: ', pruned_keras_file)

Saved pruned Keras model to:  ./baseline_pruned.h5


## Then, create a compressible model for TFLite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

pruned_tflite_file = './baseline_pruned.tflite'

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to: ', pruned_tflite_file)

Saved pruned TFLite model to:  ./baseline_pruned.tflite


## Define a helper function to accutually compress the models via gzip and measure the zipped size

In [10]:
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

## Compare and see that the models are 3x smaller from pruning

In [11]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 77986.00 bytes
Size of gzipped pruned Keras model: 25640.00 bytes
Size of gzipped pruned TFlite model: 24439.00 bytes


# Create a 10x smaller model from combining pruning and quantization

you can apply post-training quantization to the pruned model for additional benefits

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = './baseline_pruned_quantized.tflite'

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to: ', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

Saved quantized and pruned TFLite model to:  ./baseline_pruned_quantized.tflite
Size of gzipped baseline Keras model: 77986.00 bytes
Size of gzipped pruned and quantized TFlite model: 7763.00 bytes


# See persistence of accuracy from TF to TFLite

## Define a helper function to evaluate the TF Lite model on the test dataset

In [13]:
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]['index']
    output_index = interpreter.get_output_details()[0]['index']
    
    # Run predictions on ever y image in the test dataset
    prediction_digits = []
    for i, test_image in enumerate(test_images):
        # Pre-processing: add batch dimension and convert to float32 to match with the model's input data format
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        
        # Run inference
        interpreter.invoke()
        
        # Post-processing: remove batch dimension and find the digit with highest probability
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
    
    # Compare prediction results with ground truth labels to calculate accuracy
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

## you evaluate the pruned and quantized model and see that the accuracy from tensorflow persists to the TFLite backend

In [14]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Pruned and quantized TFLite test_accuracy: 0.9709
Pruned TF test accuracy: 0.9713000059127808
